*Note: this notebook was generated by dsflow*

# meteo_bretagne / create_table

## \[1- config\] get task configuration as defined in dag_specs.yaml

In [2]:
import os

ds = os.environ.get('INPUT_PARAMETER', "2017-09-23")

# Get task configuration, as defined in dag_specs.yaml
task_config={'source_path': "/data/raw/meteo/ds=%s" % ds,
             'sink_path': "/data/tables/meteo/ds=%s" % ds,
             'ds': ds}
print(task_config)

{'source_path': '/data/raw/meteo/ds=2017-09-23', 'sink_path': '/data/tables/meteo/ds=2017-09-23', 'ds': '2017-09-23'}


In [3]:
# Get task variables:
source_path = task_config["source_path"]
sink_path = task_config["sink_path"]
ds = task_config["ds"]  # ds is the execution date

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [7]:
# https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/information/

## \[2- first data check\] using Spark, print a couple lines from source_path

In [8]:
raw_data = spark.read.text(source_path)

## \[3- process data\] convert raw data into a Spark dataframe

In [9]:
dataset = (spark
.read
.options(wholeFile=True, inferSchema=True)
.json(source_path)
)

number of rows in dataset

In [10]:
dataset.count()

1694

In [11]:
# by default, dataframe uses an auto-detected schema
dataset.printSchema()

root
 |-- datasetid: string (nullable = true)
 |-- fields: struct (nullable = true)
 |    |-- ch: string (nullable = true)
 |    |-- cl: string (nullable = true)
 |    |-- cm: string (nullable = true)
 |    |-- cod_tend: long (nullable = true)
 |    |-- coordonnees: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- ctype1: string (nullable = true)
 |    |-- ctype2: string (nullable = true)
 |    |-- ctype3: string (nullable = true)
 |    |-- ctype4: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- dd: long (nullable = true)
 |    |-- etat_sol: string (nullable = true)
 |    |-- ff: double (nullable = true)
 |    |-- hbas: string (nullable = true)
 |    |-- hnuage1: string (nullable = true)
 |    |-- hnuage2: string (nullable = true)
 |    |-- hnuage3: string (nullable = true)
 |    |-- hnuage4: string (nullable = true)
 |    |-- n: string (nullable = true)
 |    |-- nbas: string (nullable = true)
 |    |-- nnuage1: string 

In [12]:
from pyspark.sql.functions import expr

In [13]:
dataset02 = (dataset
  .selectExpr("cast(fields.date as timestamp) as ts",
          "date(cast(fields.date as date)) as record_date",
          "fields.*")
  )

In [14]:
dataset02.printSchema()

root
 |-- ts: timestamp (nullable = true)
 |-- record_date: date (nullable = true)
 |-- ch: string (nullable = true)
 |-- cl: string (nullable = true)
 |-- cm: string (nullable = true)
 |-- cod_tend: long (nullable = true)
 |-- coordonnees: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- ctype1: string (nullable = true)
 |-- ctype2: string (nullable = true)
 |-- ctype3: string (nullable = true)
 |-- ctype4: string (nullable = true)
 |-- date: string (nullable = true)
 |-- dd: long (nullable = true)
 |-- etat_sol: string (nullable = true)
 |-- ff: double (nullable = true)
 |-- hbas: string (nullable = true)
 |-- hnuage1: string (nullable = true)
 |-- hnuage2: string (nullable = true)
 |-- hnuage3: string (nullable = true)
 |-- hnuage4: string (nullable = true)
 |-- n: string (nullable = true)
 |-- nbas: string (nullable = true)
 |-- nnuage1: string (nullable = true)
 |-- nnuage2: string (nullable = true)
 |-- nnuage3: string (nullable = true)
 |-- nnuage4: s

In [15]:
dataset02.groupby("record_date").count().show()

+-----------+-----+
|record_date|count|
+-----------+-----+
| 2017-01-06|    8|
| 2017-01-27|    8|
| 2017-02-26|    8|
| 2017-01-24|    8|
| 2017-06-29|    8|
| 2017-02-16|    8|
| 2017-07-31|    8|
| 2017-04-09|    8|
| 2017-03-28|    8|
| 2017-02-28|    8|
| 2017-06-30|    8|
| 2017-01-30|    8|
| 2017-07-06|    8|
| 2017-05-11|    8|
| 2017-02-10|    8|
| 2017-04-25|    8|
| 2017-03-19|    8|
| 2017-05-26|    8|
| 2017-01-04|    8|
| 2017-06-28|    8|
+-----------+-----+
only showing top 20 rows



## First 10 rows

In [16]:
dsflow.display(dataset02)

NameError: name 'dsflow' is not defined

In [17]:
dsflow.display(dataset02.describe())

NameError: name 'dsflow' is not defined

## \[4- write on disk\] write down the output to disk

In [18]:
final_dataset = dataset02

In [19]:
(final_dataset
 .write
 .mode("overwrite")
 .parquet(sink_path)
)